<a href="https://colab.research.google.com/github/sukhmancs/TextWizards/blob/main/translation_encoder_decoder_attention_french_english.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install ipympl

In [ ]:
# UPDATED
###########################################################
#!pip install gensim==4.3.1
# The library has been archived and won't be used anymore
# # !pip install allennlp==0.9.0
#!pip install flair==0.12.2
#!pip install torchvision==0.15.1
# # HuggingFace
!pip install transformers==4.32.0
!pip install datasets==2.14.4
###########################################################

In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
#%matplotlib inline
%matplotlib inline

# **IMPORT**

In [ ]:
'''
try:
    import google.colab
    import requests
    url = 'https://raw.githubusercontent.com/dvgodoy/PyTorchStepByStep/master/config.py'
    r = requests.get(url, allow_redirects=True)
    open('config.py', 'wb').write(r.content)
except ModuleNotFoundError:
    pass

from config import *
config_chapter9()
# This is needed to render the plots in this chapter
from plots.chapter8 import *
from plots.chapter9 import *
'''

"\ntry:\n    import google.colab\n    import requests\n    url = 'https://raw.githubusercontent.com/dvgodoy/PyTorchStepByStep/master/config.py'\n    r = requests.get(url, allow_redirects=True)\n    open('config.py', 'wb').write(r.content)\nexcept ModuleNotFoundError:\n    pass\n\nfrom config import *\nconfig_chapter9()\n# This is needed to render the plots in this chapter\nfrom plots.chapter8 import *\nfrom plots.chapter9 import *\n"

In [ ]:
import copy
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset, RandomSampler
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

#from data_generation.square_sequences import generate_sequences
#from stepbystep.v4 import StepByStep

In [ ]:
#!pip install trax

In [ ]:
'''
from termcolor import colored
import random
import numpy as np

import trax
from trax import layers as tl
from trax.fastmath import numpy as fastnp
from trax.supervised import training

!pip list | grep trax
'''

'\nfrom termcolor import colored\nimport random\nimport numpy as np\n\nimport trax\nfrom trax import layers as tl\nfrom trax.fastmath import numpy as fastnp\nfrom trax.supervised import training\n\n!pip list | grep trax\n'

# __Start here:__

In [ ]:
cd /content/drive/MyDrive/Code_files_and_data/data

/content/drive/MyDrive/Code_files_and_data/data


In [ ]:
!ls

data						    projections
data.csv					    question_answering
data_vie.zip					    question_answering_BERT_transformer
data.zip					    question_duplicates_Siamese_network_using_trax
ende_32k.subword				    shakespeare_data.txt
en_embeddings.p					    shakespeare.txt
en-fr.test.txt					    submission.csv
en-fr.train.txt					    surname_classification
en_US.twitter.txt				    test.csv
finetune_bert					    titanic_data
fr_embeddings.p					    train.csv
machine_translation_encoder_decoder_with_attention  transformer_summerizer_trax
model						    utils.py
model.pkl.gz					    vie-eng.zip
named_entity_recognition			    world-happiness-report-2021.csv
ner_dataset.csv					    world-happiness-report.csv
Password_Strength_Classifier_Dataset.zip


In [ ]:
# Download the data from this pytorch tutorial https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#loading-data-files
# or Download directly by following this link https://download.pytorch.org/tutorial/data.zip
import os

#zip_file_path = '/content/drive/MyDrive/Code_files_and_data/data/data.zip' # Uncomment for For french to english dataset
zip_file_path = '/content/drive/MyDrive/Code_files_and_data/data/data_vie.zip'  # Uncomment for Vietnamese to english dataset
extracted_dir_path = '/content/drive/MyDrive/Code_files_and_data/data/data/'

# Check if the extracted directory already exists
if not os.path.exists(extracted_dir_path):
    # If not, then unzip the file
    !unzip $zip_file_path
else:
    print(f"The directory {extracted_dir_path} already exists. No need to extract.")


The directory /content/drive/MyDrive/Code_files_and_data/data/data/ already exists. No need to extract.


# Prepare data

### Generate Only the data (Non preprocessed embeddings steps)

In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]  # Remove me - To train on entire data change this `for l in lines[:5000]` to `for l in lines`

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
['c est une tres gentille fille', 'she s a very nice girl']


In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

In [ ]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 10

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
input_lang, output_lang, train_dataloader = get_dataloader(batch_size=32)

for data in train_dataloader:
    inputs, targets = data
    print(inputs)
    break

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
tensor([[   2, 3493,    2,  244,   30,  245,   99, 3878,    1,    0],
        [   2,    3, 1622,   62,    1,    0,    0,    0,    0,    0],
        [   2,    7,  110,   13,    1,    0,    0,    0,    0,    0],
        [ 116,  213, 3687,  244,   21,  527,  245, 1888,    1,    0],
        [ 116,  244,   10,  821,  245,  198,    1,    0,    0,    0],
        [   2,   23, 1656,  901,  116, 2333, 2724,    1,    0,    0],
        [ 116,  213,  119, 2490,    1,    0,    0,    0,    0,    0],
        [  29,   30,  261,  232, 1581, 2166,  396, 2106,  117,    1],
        [ 116,  213,  119, 1803,    1,    0,    0,    0,    0,    0],
        [ 116,  213,   64,  855,   99,  227,    1,    0,    0,    0],
        [ 143,   21,  297, 1633,  744,    1,    0,    0,    0,    0],
        [ 121,  124, 2445,  113, 2365,  193, 3600,    1,    0,    0],
        [ 117,   21, 2074,  178, 

### PreProcessed Embeddings (old steps)

In [ ]:
english_sentences, french_sentences = zip(*pairs)

print("English sentences:", list(english_sentences))
print("French sentences:", list(french_sentences))

English sentences: ['je vais bien', 'ca va', 'je suis gras', 'je suis gros', 'je suis en forme', 'je suis touche !', 'je suis touchee !', 'je suis malade', 'je suis triste', 'je suis timide', 'je suis mouille', 'je suis mouillee', 'il est mouille', 'je suis gras', 'je suis revenu', 'me revoila', 'je suis chauve', 'je suis occupe', 'je suis occupee', 'je suis calme', 'j ai froid', 'j en ai fini', 'tout va bien', 'je vais bien', 'ca va', 'je suis libre !', 'je suis libre', 'je suis disponible', 'je suis repu !', 'je suis rassasie !', 'je suis content', 'je suis chez moi', 'je suis en retard', 'je suis paresseux', 'je suis faineant', 'je suis paresseuse', 'je suis faineante', 'je vais bien', 'je me porte bien', 'je suis en securite', 'je suis malade', 'j en suis certain', 'je suis certain', 'j en suis sur', 'j en suis sure', 'je suis grande', 'je suis mince', 'je suis ordonne', 'je suis ordonnee', 'je suis laid', 'je suis laide', 'je suis faible', 'je vais bien', 'je me porte bien', 'il e

In [ ]:
#from flair.data import Sentence

In [ ]:
# from flair.embeddings import TransformerWordEmbeddings
# bert = TransformerWordEmbeddings('bert-base-uncased', layers='-1')

In [ ]:
# def get_embeddings(embeddings, sentence):
#     sent = Sentence(sentence)
#     embeddings.embed(sent)
#     return torch.stack([token.embedding for token in sent.tokens]).float()

In [ ]:
#train_dataset_doc = [get_embeddings(bert, row) for row in english_sentences]

In [ ]:
#torch.stack(train_dataset_doc) # Embeddings for each word in a sentence
# ToDo: these embeddings have different sequence length because each sentence had different length, so do something about it.

### Preproccessing embedding (New steps)-
Only run this cell if you are using **Encoder Decoder using preprocessed embeddings cell**

In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
from transformers import BertModel, BertTokenizer
model_bert = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True,)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
MAX_LENGTH = 10

In [ ]:
text = "he eventually sold the shares back to the bank at a premium."
marked_text = "[CLS] " + text + " [SEP]"
tokenized_text_1 = tokenizer.tokenize(marked_text)
tokenized_text = tokenizer(text, max_length=MAX_LENGTH, padding="max_length", truncation=True, return_tensors="pt") # automatically adds [CLS] and [SEP] tokens
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text_1)

print(f"tokenized_text: {tokenized_text_1}, \ntokenized_text: {tokenized_text['input_ids']}, \ntokenized_text_1 indexed tokens: {indexed_tokens}")

tokenized_text: ['[CLS]', 'he', 'eventually', 'sold', 'the', 'shares', 'back', 'to', 'the', 'bank', 'at', 'a', 'premium', '.', '[SEP]'], 
tokenized_text: tensor([[ 101, 2002, 2776, 2853, 1996, 6661, 2067, 2000, 1996,  102]]), 
tokenized_text_1 indexed tokens: [101, 2002, 2776, 2853, 1996, 6661, 2067, 2000, 1996, 2924, 2012, 1037, 12882, 1012, 102]


In [ ]:
def bert_text_preparation(text, tokenizer):
  """
  Preprocesses text input in a way that BERT can interpret.
  """
  #marked_text = "[CLS] " + text + " [SEP]"
  #tokenized_text = tokenizer.tokenize(marked_text)
  #indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  #segments_ids = [1]*len(indexed_tokens)
  # convert inputs to tensors
  #tokens_tensor = torch.tensor([indexed_tokens])
  tokens_tensor = tokenizer(text, max_length=MAX_LENGTH, padding="max_length", truncation=True, return_tensors="pt") # automatically adds [CLS] and [SEP] tokens
  #segments_tensor = torch.tensor([segments_ids])
  return tokens_tensor['input_ids']

In [ ]:
def get_bert_embeddings(tokens_tensor, model):
    """
    Obtains BERT embeddings for tokens.
    """
    # gradient calculation id disabled
    with torch.no_grad():
      # obtain hidden states
      outputs = model(tokens_tensor)
      hidden_states = outputs[2]    # concatenate the tensors for all layers
    # use "stack" to create new dimension in tensor
    token_embeddings = torch.stack(hidden_states, dim=0)    # remove dimension 1, the "batches"
    token_embeddings = torch.squeeze(token_embeddings, dim=1)    # swap dimensions 0 and 1 so we can loop over tokens
    #print(f"token_embeddings_stacked_squeezed size: {token_embeddings.size()}")
    token_embeddings = token_embeddings.permute(1,0,2)    # intialized list to store embeddings
    token_vecs_sum = []    # "token_embeddings" is a [Y x 12 x 768] tensor
    # where Y is the number of tokens in the sentence    # loop over tokens in sentence
    for token in token_embeddings:    # "token" is a [12 x 768] tensor    # sum the vectors from the last four layers
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum.append(sum_vec)
    return token_vecs_sum

In [ ]:
sentence = "he eventually sold the shares back to the bank at a premium."
tokens_tensor = bert_text_preparation(sentence, tokenizer)
with torch.no_grad():
      # obtain hidden states
      outputs = model_bert(tokens_tensor)
      hidden_states = outputs[2]
# use "stack" to create new dimension in tensor
token_embeddings = torch.stack(hidden_states, dim=0)    # remove dimension 1, the "batches"
token_embeddings = torch.squeeze(token_embeddings, dim=1)    # swap dimensions 0 and 1 so we can loop over tokens
#print(f"token_embeddings_stacked_squeezed size: {token_embeddings.size()}")
token_embeddings = token_embeddings.permute(1,0,2)
token_vecs_sum = []    # "token_embeddings" is a [Y x 12 x 768] tensor
    # where Y is the number of tokens in the sentence    # loop over tokens in sentence
for token in token_embeddings:    # "token" is a [12 x 768] tensor    # sum the vectors from the last four layers
    sum_vec = torch.sum(token[-4:], dim=0)
    token_vecs_sum.append(sum_vec)
print(len(token_vecs_sum)) # A list of token embeddings for each sentence

10


In [ ]:
sentences = ["bank",
         "he eventually sold the shares back to the bank at a premium.",
         "the bank strongly resisted cutting interest rates.",
         "the bank will supply and buy back foreign currency.",
         "the bank is pressing us for repayment of the loan.",
         "the bank left its lending rates unchanged.",
         "the river flowed over the bank.",
         "tall, luxuriant plants grew along the river bank.",
         "his soldiers were arrayed along the river bank.",
         "wild flowers adorned the river bank.",
         "two fox cubs romped playfully on the river bank.",
         "the jewels were kept in a bank vault.",
         "you can stow your jewellery away in the bank.",
         "most of the money was in storage in bank vaults.",
         "the diamonds are shut away in a bank vault somewhere.",
         "thieves broke into the bank vault.",
         "can I bank on your support?",
         "you can bank on him to hand you a reasonable bill for your services.",
         "don't bank on your friends to help you out of trouble.",
         "you can bank on me when you need money.",
         "i bank on your help."
         ]

from collections import OrderedDict
context_embeddings = []
context_tokens = []
count = 0

for sentence in sentences:
  tokens_tensor = bert_text_preparation(sentence, tokenizer)
  #print(f"tokens_tensor: {tokens_tensor}")
  list_token_embeddings = get_bert_embeddings(tokens_tensor, model_bert)  # make ordered dictionary to keep track of the position of each   word
  context_embeddings.append(list_token_embeddings)
  #print(f"list_token_embedding: {list_token_embeddings}")

  # Remove me
  #if count == 2: break
  #count += 1

#   tokens = OrderedDict()  # loop over tokens in sensitive sentence
#   for token in tokenized_text[1:-1]:
#     # keep track of position of word and whether it occurs multiple times
#     if token in tokens:
#       tokens[token] += 1
#     else:
#       tokens[token] = 1  # compute the position of the current token
#     token_indices = [i for i, t in enumerate(tokenized_text) if t == token]
#     current_index = token_indices[tokens[token]-1]  # get the corresponding embedding
#     token_vec = list_token_embeddings[current_index]

#     # save values
#     context_tokens.append(token)
#    context_embeddings.append(list_token_embeddings)

In [ ]:
# Convert the list of lists of tensors to a three-dimensional tensor
context_embeddings_tensor = torch.stack([torch.stack(embeddings) for embeddings in context_embeddings], dim=0)

print(context_embeddings_tensor.shape)  # This will be of shape (N, L, H)

torch.Size([21, 10, 768])


In [ ]:
input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
['je suis un etranger ici', 'i m a stranger here']


In [ ]:
inputs = [pair[0] for pair in pairs]
print(inputs[:40])

['je vais bien', 'ca va', 'je suis gras', 'je suis gros', 'je suis en forme', 'je suis touche !', 'je suis touchee !', 'je suis malade', 'je suis triste', 'je suis timide', 'je suis mouille', 'je suis mouillee', 'il est mouille', 'je suis gras', 'je suis revenu', 'me revoila', 'je suis chauve', 'je suis occupe', 'je suis occupee', 'je suis calme', 'j ai froid', 'j en ai fini', 'tout va bien', 'je vais bien', 'ca va', 'je suis libre !', 'je suis libre', 'je suis disponible', 'je suis repu !', 'je suis rassasie !', 'je suis content', 'je suis chez moi', 'je suis en retard', 'je suis paresseux', 'je suis faineant', 'je suis paresseuse', 'je suis faineante', 'je vais bien', 'je me porte bien', 'je suis en securite']


In [ ]:
inputs = [pair[0] for pair in pairs]
context_embeddings = []
context_tokens = []
count = 0

for sentence in inputs[:20]:
    tokens_tensor = bert_text_preparation(sentence, tokenizer)
        #print(f"tokens_tensor: {tokens_tensor}")
    list_token_embeddings = get_bert_embeddings(tokens_tensor, model)  # make ordered dictionary to keep track of the position of each   word
    context_embeddings.append(list_token_embeddings)

context_embeddings_tensor = torch.stack([torch.stack(embeddings) for embeddings in context_embeddings], dim=0)
    #print(f"context_embeddings_tensor: {context_embeddings_tensor}")
train_data = TensorDataset(context_embeddings_tensor.to(device))
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=4)

In [ ]:
for data in train_dataloader:
    print(data[0].size())
    break

torch.Size([4, 10, 768])


In [ ]:
def generate_embeddings(n_pairs=None):

    #input_lang, output_lang, pairs = readLangs('eng', 'fra', True)
    input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

    if n_pairs is not None:
        pairs = pairs[:n_pairs]  # remove me

    # Extract French sentences into a separate list
    #input_sentences = [pair[0] for pair in pairs]
    #target_sentences = [pair[1] for pair in pairs]

    context_embeddings = []
    input_context_tokens = []
    target_context_tokens = []
    count = 0

    for input_sentence, target_sentence in pairs:
        input_tokens_tensor = bert_text_preparation(input_sentence, tokenizer)
        target_tokens_tensor = bert_text_preparation(target_sentence, tokenizer)
        #print(f"tokens_tensor: {tokens_tensor}")
        list_token_embeddings = get_bert_embeddings(input_tokens_tensor, model)  # make ordered dictionary to keep track of the position of each   word
        #print(f"input_tokens_tensor:\n{input_tokens_tensor},\ntarget_tokens_tensor:\n{target_tokens_tensor}")
        context_embeddings.append(list_token_embeddings)
        input_context_tokens.append(input_tokens_tensor)
        target_context_tokens.append(target_tokens_tensor)

    context_embeddings_tensor = torch.stack([torch.stack(embeddings) for embeddings in context_embeddings], dim=0)

    # Convert the list of tensors to a three-dimensional tensor
    input_context_tokens_tensor = torch.stack(input_context_tokens, dim=0)
    target_context_tokens_tensor = torch.stack(target_context_tokens, dim=0)

    # Remove the middle dimension with size 1
    input_context_tokens_tensor = input_context_tokens_tensor.squeeze(dim=1)
    target_context_tokens_tensor = target_context_tokens_tensor.squeeze(dim=1)

    return context_embeddings_tensor, input_context_tokens_tensor, target_context_tokens_tensor

In [ ]:
context_embeddings_tensor, input_context_tokens_tensor, target_context_tokens_tensor = generate_embeddings(n_pairs=10)

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991


In [ ]:
print(f"input_context_tokens_tensor:\n{input_context_tokens_tensor}, \ntarget_context_tokens_tensor:\n{target_context_tokens_tensor}")

input_context_tokens_tensor:
tensor([[  101, 15333, 12436,  2483, 29316,   102,     0,     0,     0,     0],
        [  101,  6187, 12436,   102,     0,     0,     0,     0,     0,     0],
        [  101, 15333, 24086,  2015, 24665,  3022,   102,     0,     0,     0],
        [  101, 15333, 24086,  2015, 24665,  2891,   102,     0,     0,     0],
        [  101, 15333, 24086,  2015,  4372,  2433,  2063,   102,     0,     0],
        [  101, 15333, 24086,  2015,  3543,  2063,   999,   102,     0,     0],
        [  101, 15333, 24086,  2015,  3543,  4402,   999,   102,     0,     0],
        [  101, 15333, 24086,  2015, 28935,  3207,   102,     0,     0,     0],
        [  101, 15333, 24086,  2015, 13012, 13473,   102,     0,     0,     0],
        [  101, 15333, 24086,  2015,  5199,  5178,   102,     0,     0,     0]]), 
target_context_tokens_tensor:
tensor([[  101,  1045,  1049,  7929,   102,     0,     0,     0,     0,     0],
        [  101,  1045,  1049,  7929,   102,     0,     0, 

In [ ]:
#@title Preprocessed embeddings dataloader

def get_dataloader(batch_size, n_pairs=None):
    context_embeddings_tensor, input_context_tokens_tensor, target_context_tokens_tensor = generate_embeddings(n_pairs)

    print("worked so far")
    #target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    # Extract French sentences into a separate list
    #input_sentences = [pair[0] for pair in pairs]

    #sentences = []
    # for idx, (inp, tgt) in enumerate(pairs):
    #     #inp_ids = indexesFromSentence(input_lang, inp)
    #     tgt_ids = indexesFromSentence(output_lang, tgt)
    #     #inp_ids.append(EOS_token)
    #     #sentences.append(inp)
    #     tgt_ids.append(EOS_token)
    #     #input_ids[idx, :len(inp_ids)] = inp_ids
    #     target_ids[idx, :len(tgt_ids)] = tgt_ids

    # context_embeddings = []
    # context_tokens = []
    # count = 0

    # for sentence in input_sentences:
    #     tokens_tensor = bert_text_preparation(sentence, tokenizer)
    #     #print(f"tokens_tensor: {tokens_tensor}")
    #     list_token_embeddings = get_bert_embeddings(tokens_tensor, model)  # make ordered dictionary to keep track of the position of each   word
    #     context_embeddings.append(list_token_embeddings)

    # context_embeddings_tensor = torch.stack([torch.stack(embeddings) for embeddings in context_embeddings], dim=0)
    #print(f"context_embeddings_tensor: {context_embeddings_tensor}")
    #train_data = TensorDataset(context_embeddings_tensor.to(device),
     #                          torch.LongTensor(target_ids.view(-1)).to(device))
    train_data = TensorDataset(input_context_tokens_tensor.to(device), target_context_tokens_tensor.to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader, context_embeddings_tensor.view(-1, context_embeddings_tensor.size(-1))

In [ ]:
input_lang, output_lang, train_dataloader, context_embeddings_tensor = get_dataloader(batch_size=4, n_pairs=10)

for data in train_dataloader:
    inputs, targets = data
    print(f"inputs type: {type(inputs)}, \ntargets types: {type(targets)}")
    print(f"inputs size: {inputs.size()}, \ntargets size: {targets.size()}")
    print(f"input_tokens:\n{inputs}, \ntarget_tokens:\n{targets}")
    #print(f"last input dimension: {inputs.size(-1)}")
    #print(f"reshaped input dimension: {inputs.view(-1, inputs.size(-1)).size()}")
    break

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
worked so far
inputs type: <class 'torch.Tensor'>, 
targets types: <class 'torch.Tensor'>
inputs size: torch.Size([4, 10]), 
targets size: torch.Size([4, 10])
input_tokens:
tensor([[  101, 15333, 24086,  2015,  4372,  2433,  2063,   102,     0,     0],
        [  101, 15333, 12436,  2483, 29316,   102,     0,     0,     0,     0],
        [  101, 15333, 24086,  2015,  3543,  2063,   999,   102,     0,     0],
        [  101, 15333, 24086,  2015, 13012, 13473,   102,     0,     0,     0]]), 
target_tokens:
tensor([[ 101, 1045, 1049, 4906,  102,    0,    0,    0,    0,    0],
        [ 101, 1045, 1049, 7929,  102,    0,    0,    0,    0,    0],
        [ 101, 1045, 1049, 2718,  999,  102,    0,    0,    0,    0],
        [ 101, 1045, 1049, 6517,  102,    0,    0,    0,    0,    0]])


In [ ]:
# import os
# filepath = os.path.join('/content/drive/MyDrive/Code_files_and_data/data/projections/')
# name = 'metadata_small.tsv'
# with open(os.path.join(filepath, name), 'w+') as file_metadata:
#   for i, token in enumerate(context_tokens):
#     file_metadata.write(token + '\n')

# import csv
# name = 'embeddings_small.tsv'
# with open(os.path.join(filepath, name), 'w+') as tsvfile:
#     writer = csv.writer(tsvfile, delimiter='\t')
#     for embedding in context_embeddings:
#         writer.writerow(embedding.numpy())

# ## Use the data stored in /content/drive/MyDrive/Code_files_and_data/data/projections/ folder and upload it to tensorboard to visualize

# Encoder-Decoder with Attention and RNN
Note: I have also included code for simple Decoder that do not use attention.

## Encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, n_features, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_features = n_features
        self.hidden = None
        #self.embedding_dimension = 256
        self.embd = nn.Embedding(self.n_features, self.hidden_dim)
        self.basic_rnn = nn.GRU(self.hidden_dim, self.hidden_dim, batch_first=True)

    def forward(self, X):
        X_embd = self.embd(X) # N, F -> N, F, H
        rnn_out, self.hidden = self.basic_rnn(X_embd) # N, F, H x N, H, H  ->  N, F, H

        return rnn_out, self.hidden # N, F, H

In [ ]:
embd = nn.Embedding(5, 5)
rnn = nn.GRU(5, 5, batch_first=True)

In [ ]:
full_seq = torch.full((2, 3), 1)
rnn_out, final_hidden = rnn(embd(full_seq))
final_hidden

tensor([[[ 0.7565,  0.5769,  0.1726,  0.0968, -0.3114],
         [ 0.7565,  0.5769,  0.1726,  0.0968, -0.3114]]],
       grad_fn=<StackBackward0>)

### Testig Encoder

In [ ]:
#torch.manual_seed(21)
full_seq = torch.full((2, 3), 1).to(device)
encoder = Encoder(n_features=3, hidden_dim=5).to(device)
hidden_seq, hidden_final = encoder(full_seq) # output is N, L, F
hidden_final

tensor([[[ 0.3413,  0.1784, -0.0800, -0.5347,  0.4507],
         [ 0.3413,  0.1784, -0.0800, -0.5347,  0.4507]]],
       grad_fn=<StackBackward0>)

## Simple Decoder (no Attention)

![](https://github.com/dvgodoy/PyTorchStepByStep/blob/master/images/decoder.png?raw=1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_size, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.hidden = None
        self.embedding = nn.Embedding(output_size, self.hidden_dim)
        self.basic_rnn = nn.GRU(self.hidden_dim, self.hidden_dim, batch_first=True)
        self.regression = nn.Linear(self.hidden_dim, output_size)

    def init_hidden(self, hidden_final):
        # We only need the final hidden state from encoder for each sentence
        #hidden_final = hidden_seq[:, -1:] # N, F
        self.hidden = hidden_final
        #self.hidden = hidden_final.permute(1, 0, 2) # 1, N, H

    def forward(self, X):
        X = self.embedding(X) # N, 1 -> N, F
        batch_first_output, self.hidden = self.basic_rnn(X, self.hidden) # N, F
        out = self.regression(batch_first_output) # N, output_size

        # N, 1, F
        return out

### Testing Decoder

In [ ]:
torch.manual_seed(21)
decoder = Decoder(output_size=5, hidden_dim=5).to(device)
batch_size = 16

# Initial hidden state will be encoder's final hidden state
decoder.init_hidden(hidden_final)
# Initial data point is the last element of source sequence
#inputs = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
inputs = torch.empty(2, 1, dtype=torch.long, device=device).fill_(SOS_token)  # remove me

decoder_outputs = []
target_len = 2
for i in range(target_len):
    print(f'Hidden: {decoder.hidden}')
    decoder_output = decoder(inputs)   # Predicts coordinates
    decoder_outputs.append(decoder_output)
    _, topi = decoder_output.topk(1)
    inputs = topi.squeeze(-1).detach() # detach from history as input
    print(f'Output: {decoder_output}\n')
decoder_outputs = torch.cat(decoder_outputs, dim=1)
print(f'combinet_outputs: {decoder_outputs}')

Hidden: tensor([[[-0.0072,  0.9993, -0.0021,  0.0000,  0.8425],
         [-0.0072,  0.9993, -0.0021,  0.0000,  0.8425]]],
       grad_fn=<StackBackward0>)
Output: tensor([[[ 0.6876, -0.0527, -0.0652,  0.1395, -0.3921]],

        [[ 0.6876, -0.0527, -0.0652,  0.1395, -0.3921]]],
       grad_fn=<ViewBackward0>)

Hidden: tensor([[[ 0.2679,  0.6601, -0.1505, -0.2759,  0.4169],
         [ 0.2679,  0.6601, -0.1505, -0.2759,  0.4169]]],
       grad_fn=<StackBackward0>)
Output: tensor([[[ 0.6594,  0.0165,  0.0516,  0.1572, -0.3371]],

        [[ 0.6594,  0.0165,  0.0516,  0.1572, -0.3371]]],
       grad_fn=<ViewBackward0>)

combinet_outputs: tensor([[[ 0.6876, -0.0527, -0.0652,  0.1395, -0.3921],
         [ 0.6594,  0.0165,  0.0516,  0.1572, -0.3371]],

        [[ 0.6876, -0.0527, -0.0652,  0.1395, -0.3921],
         [ 0.6594,  0.0165,  0.0516,  0.1572, -0.3371]]],
       grad_fn=<CatBackward0>)


## Decoder with attention

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_dim, input_dim=None, proj_values=False):
        super().__init__()
        self.d_k = hidden_dim
        self.input_dim = hidden_dim if input_dim is None else input_dim
        self.proj_values = proj_values
        # Affine transformations for Q, K, and V
        self.linear_query = nn.Linear(self.input_dim, hidden_dim)
        self.linear_key = nn.Linear(self.input_dim, hidden_dim)
        self.linear_value = nn.Linear(self.input_dim, hidden_dim)
        self.alphas = None

    def init_keys(self, keys):
        self.keys = keys
        self.proj_keys = self.linear_key(self.keys) # N, F, H x N, H, H -> N, F, H
        self.values = self.linear_value(self.keys) if self.proj_values else self.keys  # N, F, H x N, H, H -> N, F, H

    def score_function(self, query):
        proj_query = self.linear_query(query) # N, 1, H x N, H, H -> N, 1, H
        # scaled dot product
        # N, 1, H x N, H, F -> N, 1, F
        dot_products = torch.bmm(proj_query, self.proj_keys.permute(0, 2, 1))
        scores =  dot_products / np.sqrt(self.d_k)
        return scores

    def forward(self, query, mask=None):
        # Query is batch-first N, 1, H
        # L or F means sequence length
        scores = self.score_function(query) # N, 1, F
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        alphas = F.softmax(scores, dim=-1) # N, 1, F
        self.alphas = alphas.detach()

        # N, 1, F x N, F, H -> N, 1, H
        context = torch.bmm(alphas, self.values)
        return context

### Decoder with rnn and attention

In [ ]:
class DecoderAttn(nn.Module):
    def __init__(self, output_size, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.hidden = None
        self.output_size = output_size
        self.embedding = nn.Embedding(output_size, self.hidden_dim)
        self.basic_rnn = nn.GRU(self.hidden_dim, self.hidden_dim, batch_first=True)
        self.attn = Attention(self.hidden_dim)
        self.regression = nn.Linear(2 * self.hidden_dim, self.output_size)

    def init_hidden(self, hidden_seq):
        # the output of the encoder is N, F, H
        # and init_keys expects batch-first as well
        self.attn.init_keys(hidden_seq)
        hidden_final = hidden_seq[:, -1:]
        self.hidden = hidden_final.permute(1, 0, 2)   # F, N, H

    def forward(self, X, mask=None):
        # X is N, 1
        # N is batch size, H is hidden dimensions
        X = self.embedding(X) # N, 1 -> N, 1, H
        batch_first_output, self.hidden = self.basic_rnn(X, self.hidden) # N, 1, H x N, H, H -> N, 1, H

        query = batch_first_output # N, 1, H
        # Attention
        context = self.attn(query, mask=mask) # N, 1, H
        concatenated = torch.cat([context, query], axis=-1) # N, 1, 2*H
        out = self.regression(concatenated)  # N, 1, 2*H x N, 2*H, 1 -> N, 1, 1

        # N, 1, F
        return out.view(-1, 1, self.output_size)

### Testing Decoder with attention

In [ ]:
inputs = torch.empty(2, 1, dtype=torch.long, device=device).fill_(SOS_token)  # remove me

In [ ]:
embedding = nn.Embedding(1, 5).to(device)
embedding(inputs).size()

torch.Size([2, 1, 5])

In [ ]:
#torch.manual_seed(21)
decoder = DecoderAttn(output_size=5, hidden_dim=5)

# Initial hidden state will be encoder's final hidden state
decoder.init_hidden(hidden_seq)
# Initial data point is the last element of source sequence
#inputs = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
inputs = torch.empty(2, 1, dtype=torch.long, device=device).fill_(SOS_token)  # remove me

decoder_outputs = []
target_len = 2
for i in range(target_len):
    print(f'Hidden: {decoder.hidden}')
    decoder_output = decoder(inputs)   # Predicts coordinates
    decoder_outputs.append(decoder_output)
    _, topi = decoder_output.topk(1)
    inputs = topi.squeeze(-1).detach()  # detach from history as input
    print(f'Output: {decoder_output}\n')
decoder_outputs = torch.cat(decoder_outputs, dim=1)
print(f'combinet_outputs: {decoder_outputs}')

Hidden: tensor([[[-0.0072,  0.9993, -0.0021,  0.0000,  0.8425],
         [-0.0072,  0.9993, -0.0021,  0.0000,  0.8425]]],
       grad_fn=<PermuteBackward0>)
Output: tensor([[[0.0058, 0.4841, 0.0976, 0.0269, 0.6177]],

        [[0.0058, 0.4841, 0.0976, 0.0269, 0.6177]]], grad_fn=<ViewBackward0>)

Hidden: tensor([[[ 0.4498,  0.9413, -0.2960, -0.1569,  0.5914],
         [ 0.4498,  0.9413, -0.2960, -0.1569,  0.5914]]],
       grad_fn=<StackBackward0>)
Output: tensor([[[-0.0844,  0.5769,  0.0362, -0.2356,  0.5942]],

        [[-0.0844,  0.5769,  0.0362, -0.2356,  0.5942]]],
       grad_fn=<ViewBackward0>)

combinet_outputs: tensor([[[ 0.0058,  0.4841,  0.0976,  0.0269,  0.6177],
         [-0.0844,  0.5769,  0.0362, -0.2356,  0.5942]],

        [[ 0.0058,  0.4841,  0.0976,  0.0269,  0.6177],
         [-0.0844,  0.5769,  0.0362, -0.2356,  0.5942]]],
       grad_fn=<CatBackward0>)


## Encoder-Decoder Architecture

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, target_len, teacher_forcing_prob=0.5):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.target_len = target_len
        self.teacher_forcing_prob = teacher_forcing_prob
        self.outputs = None

    def init_outputs(self, batch_size):
        device = next(self.parameters()).device
        # N, L (target), F
        self.outputs = torch.zeros(batch_size,
                              self.target_len,
                              self.encoder.n_features).to(device)

    def store_output(self, i, out):
        # Stores the output
        self.outputs[:, i:i+1, :] = out

    def forward(self, X, target_tensor=None):
        # X is batch of sentences -> N, F
        # splits the data in source and target sequences
        # the target seq will be empty in testing mode
        # N, L, F

        # Encoder expected N, F
        hidden_seq, hidden_final = self.encoder(X)
        # Output is N, F, hidden_dim
        self.decoder.init_hidden(hidden_seq)

        # The last input of the encoder is also
        # the first input of the decoder
        #dec_inputs = source_seq[:, -1:, :]
        batch_size = hidden_seq.size(0)

        dec_inputs = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_outputs = []
        # Generates as many outputs as the target length
        for i in range(self.target_len):
            # Output of decoder is N, 1, F
            decoder_output = self.decoder(dec_inputs)
            decoder_outputs.append(decoder_output)

            prob = self.teacher_forcing_prob

            # In evaluation/test the target sequence is
            # unknown, so we cannot use teacher forcing
            if not self.training:
                prob = 0

            if torch.rand(1) <= prob and target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input
            #_, topi = decoder_output.topk(1)
            #dec_inputs = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs

In [ ]:
#@title Testing encoder-decoder with attention
hidden_size = 128
batch_size = 32
encoder = Encoder(input_lang.n_words, hidden_size).to(device)
decoder = DecoderAttn(output_lang.n_words, hidden_size).to(device)

In [ ]:
hidden_seq, hidden_final = encoder(full_seq)
hidden_final.size()

torch.Size([1, 2, 128])

In [ ]:
inputs = torch.empty(2, 1, dtype=torch.long, device=device).fill_(SOS_token)
decoder.init_hidden(hidden_seq)
decoder(inputs)

tensor([[[-0.6018, -0.3310, -0.4661,  ..., -0.0548,  0.8664, -0.1108]],

        [[-0.6018, -0.3310, -0.4661,  ..., -0.0548,  0.8664, -0.1108]]],
       grad_fn=<ViewBackward0>)

In [ ]:
encdec = EncoderDecoder(encoder, decoder, target_len=5)
outputs = encdec(full_seq)

In [ ]:
outputs.size()

torch.Size([2, 5, 2991])

## Encoder-Decoder Architecture With preprocessed embeddings

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, target_len, teacher_forcing_prob=0.5):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.target_len = target_len
        self.teacher_forcing_prob = teacher_forcing_prob
        self.outputs = None

    def init_outputs(self, batch_size):
        device = next(self.parameters()).device
        # N, L (target), F
        self.outputs = torch.zeros(batch_size,
                              self.target_len,
                              self.encoder.n_features).to(device)

    def store_output(self, i, out):
        # Stores the output
        self.outputs[:, i:i+1, :] = out

    def forward(self, X, target_tensor=None):
        # X is batch of sentences -> N, F
        # splits the data in source and target sequences
        # the target seq will be empty in testing mode
        # N, L, F

        # Encoder expected N, F
        hidden_seq, hidden_final = self.encoder(X)
        # Output is N, F, hidden_dim
        self.decoder.init_hidden(hidden_seq)

        # The last input of the encoder is also
        # the first input of the decoder
        #dec_inputs = source_seq[:, -1:, :]
        batch_size = hidden_seq.size(0)

        dec_inputs = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_outputs = []
        # Generates as many outputs as the target length
        for i in range(self.target_len):
            # Output of decoder is N, 1, F
            decoder_output = self.decoder(dec_inputs)
            decoder_outputs.append(decoder_output)

            prob = self.teacher_forcing_prob

            # In evaluation/test the target sequence is
            # unknown, so we cannot use teacher forcing
            if not self.training:
                prob = 0

            if torch.rand(1) <= prob and target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input
            #_, topi = decoder_output.topk(1)
            #dec_inputs = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs

In [ ]:
#@title Testing simple encoder-decoder with preprocessed embeddings
hidden_size = 128
encoder = Encoder(context_embeddings_tensor, hidden_dim=hidden_size).to(device)
decoder = Decoder(pretrained_embeddings=context_embeddings_tensor, output_size=output_lang.n_words, hidden_dim=hidden_size).to(device)

In [ ]:
#@title Testing encoder-decoder with attention
hidden_size = 128
batch_size = 32
encoder = Encoder(input_lang.n_words, hidden_size).to(device)
decoder = DecoderAttn(output_lang.n_words, hidden_size).to(device)

In [ ]:
hidden_seq, hidden_final = encoder(full_seq)
hidden_final.size()

torch.Size([1, 2, 128])

In [ ]:
inputs = torch.empty(2, 1, dtype=torch.long, device=device).fill_(SOS_token)
decoder.init_hidden(hidden_seq)
decoder(inputs)

tensor([[[-0.6018, -0.3310, -0.4661,  ..., -0.0548,  0.8664, -0.1108]],

        [[-0.6018, -0.3310, -0.4661,  ..., -0.0548,  0.8664, -0.1108]]],
       grad_fn=<ViewBackward0>)

In [ ]:
encdec = EncoderDecoder(encoder, decoder, target_len=5)
outputs = encdec(full_seq)

In [ ]:
outputs.size()

torch.Size([2, 5, 2991])

# Encoder Decoder using preprocessed embeddings

## Encoder using preprocessed embeddings

In [ ]:
class Encoder(nn.Module):
    def __init__(self, pretrained_embeddings, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embd_dim = pretrained_embeddings.size(-1)
        #self.n_features = n_features
        self.hidden = None
        #self.embedding_dimension = 256
        #self.embd = nn.Embedding(self.n_features, self.hidden_dim)

        # Create an Embedding layer and initialize it with the reshaped embeddings
        self.embd = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=True)
        self.basic_rnn = nn.GRU(self.embd_dim, self.hidden_dim, batch_first=True)

    def forward(self, X):
        X_embd = self.embd(X) # N, F -> N, F, embd_dim
        rnn_out, self.hidden = self.basic_rnn(X_embd) # N, F, embd_dim x N, embd_dim, H  ->  N, F, H

        return rnn_out, self.hidden # N, F, H

In [ ]:
embd = nn.Embedding(5, 5)
rnn = nn.GRU(5, 5, batch_first=True)

In [ ]:
full_seq = torch.full((2, 3), 1)
rnn_out, final_hidden = rnn(embd(full_seq))
final_hidden

tensor([[[-0.3180,  0.5062,  0.4158, -0.3274,  0.6366],
         [-0.3180,  0.5062,  0.4158, -0.3274,  0.6366]]],
       grad_fn=<StackBackward0>)

### Testig Encoder with preprocessed embeddings

In [ ]:
#torch.manual_seed(21)
full_seq = torch.full((2, 3), 1).to(device)
encoder = Encoder(context_embeddings_tensor, hidden_dim=5).to(device)
hidden_seq, hidden_final = encoder(full_seq) # output is N, L, F
hidden_final

tensor([[[-0.0072,  0.9993, -0.0021,  0.0000,  0.8425],
         [-0.0072,  0.9993, -0.0021,  0.0000,  0.8425]]],
       grad_fn=<StackBackward0>)

## Simple Decoder with preprocessed embeddings

In [ ]:
class Decoder(nn.Module):
    def __init__(self, pretrained_embeddings, output_size, hidden_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embd_dim = pretrained_embeddings.size(-1)
        self.hidden = None
        #self.embedding = nn.Embedding(output_size, self.hidden_dim)

        self.embd = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=True)
        self.basic_rnn = nn.GRU(self.embd_dim, self.hidden_dim, batch_first=True)
        self.regression = nn.Linear(self.hidden_dim, output_size)

    def init_hidden(self, hidden_final):
        # We only need the final hidden state from encoder for each sentence
        #hidden_final = hidden_seq[:, -1:] # N, F
        #self.hidden = hidden_final
        #self.hidden = hidden_final.permute(1, 0, 2) # 1, N, H

        hidden_final = hidden_seq[:, -1:]
        self.hidden = hidden_final.permute(1, 0, 2)   # F, N, H

    def forward(self, X):
        X = self.embd(X) # N, 1 -> N, 1, embd_dim
        batch_first_output, self.hidden = self.basic_rnn(X, self.hidden)  # N, 1, embd_dim x N, embd_dim, H -> N, 1, H
        out = self.regression(batch_first_output) # N, 1, output_size

        # N, 1, output_size
        return out

### Testing Decoder

In [ ]:
torch.manual_seed(21)
decoder = Decoder(pretrained_embeddings=context_embeddings_tensor, output_size=5, hidden_dim=5).to(device)
batch_size = 16

# Initial hidden state will be encoder's final hidden state
decoder.init_hidden(hidden_seq)
# Initial data point is the last element of source sequence
#inputs = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
inputs = torch.empty(2, 1, dtype=torch.long, device=device).fill_(SOS_token)  # remove me

decoder_outputs = []
target_len = 2
for i in range(target_len):
    print(f'Hidden: {decoder.hidden}')
    decoder_output = decoder(inputs)   # Predicts coordinates
    decoder_outputs.append(decoder_output)
    _, topi = decoder_output.topk(1)
    inputs = topi.squeeze(-1).detach() # detach from history as input
    print(f'Output: {decoder_output}\n')
decoder_outputs = torch.cat(decoder_outputs, dim=1)
print(f'combinet_outputs: {decoder_outputs}')

Hidden: tensor([[[-0.0072,  0.9993, -0.0021,  0.0000,  0.8425],
         [-0.0072,  0.9993, -0.0021,  0.0000,  0.8425]]],
       grad_fn=<PermuteBackward0>)
Output: tensor([[[ 0.3327, -0.6985, -0.7303, -0.2521, -0.3805]],

        [[ 0.3327, -0.6985, -0.7303, -0.2521, -0.3805]]],
       grad_fn=<ViewBackward0>)

Hidden: tensor([[[-0.0072, -1.0000, -0.0033, -1.0000,  0.8425],
         [-0.0072, -1.0000, -0.0033, -1.0000,  0.8425]]],
       grad_fn=<StackBackward0>)
Output: tensor([[[ 0.3328, -0.6972, -0.7318, -0.2512, -0.3795]],

        [[ 0.3328, -0.6972, -0.7318, -0.2512, -0.3795]]],
       grad_fn=<ViewBackward0>)

combinet_outputs: tensor([[[ 0.3327, -0.6985, -0.7303, -0.2521, -0.3805],
         [ 0.3328, -0.6972, -0.7318, -0.2512, -0.3795]],

        [[ 0.3327, -0.6985, -0.7303, -0.2521, -0.3805],
         [ 0.3328, -0.6972, -0.7318, -0.2512, -0.3795]]],
       grad_fn=<CatBackward0>)


## Encoder-Decoder Architecture With preprocessed embeddings

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, target_len, teacher_forcing_prob=0.5):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.target_len = target_len
        self.teacher_forcing_prob = teacher_forcing_prob
        self.outputs = None

    def init_outputs(self, batch_size):
        device = next(self.parameters()).device
        # N, L (target), F
        self.outputs = torch.zeros(batch_size,
                              self.target_len,
                              self.encoder.n_features).to(device)

    def store_output(self, i, out):
        # Stores the output
        self.outputs[:, i:i+1, :] = out

    def forward(self, X, target_tensor=None):
        # X is batch of sentences -> N, F
        # splits the data in source and target sequences
        # the target seq will be empty in testing mode
        # N, L, F

        # Encoder expected N, F
        hidden_seq, hidden_final = self.encoder(X)
        # Output is N, F, hidden_dim
        self.decoder.init_hidden(hidden_seq)

        # The last input of the encoder is also
        # the first input of the decoder
        #dec_inputs = source_seq[:, -1:, :]
        batch_size = hidden_seq.size(0)

        dec_inputs = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_outputs = []
        # Generates as many outputs as the target length
        for i in range(self.target_len):
            # Output of decoder is N, 1, F
            decoder_output = self.decoder(dec_inputs)
            decoder_outputs.append(decoder_output)

            prob = self.teacher_forcing_prob

            # In evaluation/test the target sequence is
            # unknown, so we cannot use teacher forcing
            if not self.training:
                prob = 0

            if torch.rand(1) <= prob and target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input
            #_, topi = decoder_output.topk(1)
            #dec_inputs = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs

In [ ]:
#@title Testing simple encoder-decoder with preprocessed embeddings
hidden_size = 128
encoder = Encoder(context_embeddings_tensor, hidden_dim=hidden_size).to(device)
decoder = Decoder(pretrained_embeddings=context_embeddings_tensor, output_size=output_lang.n_words, hidden_dim=hidden_size).to(device)

In [ ]:
hidden_seq, hidden_final = encoder(full_seq)
hidden_final.size()

torch.Size([1, 2, 128])

In [ ]:
inputs = torch.empty(2, 1, dtype=torch.long, device=device).fill_(SOS_token)
decoder.init_hidden(hidden_seq)
decoder(inputs)

tensor([[[ 0.2513,  0.3308, -0.0428,  ..., -0.3892, -0.1224, -0.1972]],

        [[ 0.2513,  0.3308, -0.0428,  ..., -0.3892, -0.1224, -0.1972]]],
       grad_fn=<ViewBackward0>)

In [ ]:
encdec = EncoderDecoder(encoder, decoder, target_len=5)
outputs = encdec(full_seq)

In [ ]:
outputs.size()

torch.Size([2, 5, 2991])

# Encoder-Decoder with Self Attention (Most Advanced Model) (WITHOUT RNN)
It is not working, and is giving me repetitive predictions.
---   
**_I pity the fool using RNN_**

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, d_model, input_dim=None, proj_values=True):
        super().__init__()
        self.linear_out = nn.Linear(n_heads * d_model, d_model)
        self.attn_heads = nn.ModuleList([Attention(d_model,
                                                   input_dim=input_dim,
                                                   proj_values=proj_values)
                                         for _ in range(n_heads)])

    def init_keys(self, key):
        for attn in self.attn_heads:
            attn.init_keys(key)

    @property
    def alphas(self):
        # Shape: n_heads, N, 1, L (source)
        return torch.stack([attn.alphas for attn in self.attn_heads], dim=0)

    def output_function(self, contexts):
        # N, 1, n_heads * D
        concatenated = torch.cat(contexts, axis=-1)
        # Linear transf. to go back to original dimension
        out = self.linear_out(concatenated) # N, 1, D
        return out

    def forward(self, query, mask=None):
        contexts = [attn(query, mask=mask) for attn in self.attn_heads]
        out = self.output_function(contexts)
        return out

### Encoder with self attention

In [ ]:
class EncoderSelfAttn(nn.Module):
    def __init__(self, n_features, d_model, n_heads, ff_units):
        super().__init__()
        self.d_model = d_model  # d_model is just another name for hidden_dim
        self.ff_units = ff_units
        self.n_features = n_features
        self.n_heads = n_heads
        self.hidden = None
        self.embd = nn.Embedding(self.n_features, self.d_model)
        self.self_attn_heads = MultiHeadAttention(n_heads, d_model, input_dim=self.d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, ff_units),
            nn.ReLU(),
            nn.Linear(ff_units, d_model),
        )

    def forward(self, X):
        # X is source sequence of shape N, F
        X_embd = self.embd(X) # N, F -> N, F, H
        #rnn_out, self.hidden = self.basic_rnn(X_embd) # N, F, H x N, H, H  ->  N, F, H

        self.self_attn_heads.init_keys(X_embd)
        att = self.self_attn_heads(X_embd) # N, F, n_heads * H
        out = self.ffn(att)  # N, F, H

        return out # N, F, H

In [ ]:
inputs = torch.empty(2, 2, dtype=torch.long, device=device).fill_(SOS_token)  # remove me

In [ ]:
encoderSelfAttention = EncoderSelfAttn(n_features=2, d_model=5, n_heads=4, ff_units=4).to(device)
hidden_seq = encoderSelfAttention(inputs)
hidden_seq

tensor([[[ 0.4564, -0.2936,  0.2393, -0.3333,  0.3368],
         [ 0.4564, -0.2936,  0.2393, -0.3333,  0.3368]],

        [[ 0.4564, -0.2936,  0.2393, -0.3333,  0.3368],
         [ 0.4564, -0.2936,  0.2393, -0.3333,  0.3368]]],
       grad_fn=<ViewBackward0>)

### Decoder with self attention and cross attention

In [ ]:
class DecoderSelfAttn(nn.Module):
    def __init__(self, n_features, d_model, n_heads, ff_units):
        super().__init__()
        self.d_model = d_model
        self.hidden = None
        self.n_features = n_features
        self.embedding = nn.Embedding(self.n_features, self.d_model)
        self.self_attn_heads = MultiHeadAttention(n_heads, self.d_model, input_dim=self.d_model)
        self.cross_attn_heads = MultiHeadAttention(n_heads, self.d_model)
        self.ffn = nn.Sequential(
            nn.Linear(2*d_model, ff_units),
            nn.ReLU(),
            nn.Linear(ff_units, self.n_features),
        )

    def init_hidden(self, hidden_seq):
        # the output of the encoder is N, F, H
        # and init_keys expects batch-first as well
        self.cross_attn_heads.init_keys(hidden_seq)

    def forward(self, X, mask=None):
        # X is N, 1
        # N is batch size, H is hidden dimensions
        X_embd = self.embedding(X) # N, 1 -> N, 1, H
        self.self_attn_heads.init_keys(X_embd)
        attn = self.self_attn_heads(X_embd)  # N, 1, H
        attn2 = self.cross_attn_heads(attn)  # N, 1, H

        concatenated = torch.cat([attn2, X_embd], axis=-1) # N, 1, 2*H
        #out = self.regression(concatenated)  # N, 1, 2*H x N, 2*H, 1 -> N, 1, 1
        out = self.ffn(concatenated) # N, 1, n_features

        # N, 1, F
        return out

In [ ]:
torch.manual_seed(21)
decoder = DecoderSelfAttn(n_features=5, d_model=5, n_heads=5, ff_units=5)

# Initial hidden state will be encoder's final hidden state
decoder.init_hidden(hidden_seq)
# Initial data point is the last element of source sequence
#inputs = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
inputs = torch.empty(2, 1, dtype=torch.long, device=device).fill_(SOS_token)  # remove me

decoder_outputs = []
target_len = 2
for i in range(target_len):
    decoder_output = decoder(inputs)   # Predicts coordinates
    decoder_outputs.append(decoder_output)
    _, topi = decoder_output.topk(1)
    inputs = topi.squeeze(-1).detach()  # detach from history as input
    print(f'Output: {decoder_output}\n')
decoder_outputs = torch.cat(decoder_outputs, dim=1)
print(f'combinet_outputs: {decoder_outputs}')

Output: tensor([[[ 0.3530, -0.2214,  0.4408, -0.2818,  0.0044]],

        [[ 0.3530, -0.2214,  0.4408, -0.2818,  0.0044]]],
       grad_fn=<ViewBackward0>)

Output: tensor([[[ 0.7678, -0.5111,  0.8152, -0.1618, -0.1201]],

        [[ 0.7678, -0.5111,  0.8152, -0.1618, -0.1201]]],
       grad_fn=<ViewBackward0>)

combinet_outputs: tensor([[[ 0.3530, -0.2214,  0.4408, -0.2818,  0.0044],
         [ 0.7678, -0.5111,  0.8152, -0.1618, -0.1201]],

        [[ 0.3530, -0.2214,  0.4408, -0.2818,  0.0044],
         [ 0.7678, -0.5111,  0.8152, -0.1618, -0.1201]]],
       grad_fn=<CatBackward0>)


## Encoder-Decoder Self-Attention

In [ ]:
class EncoderDecoderSelfAttention(nn.Module):
    def __init__(self, encoder, decoder, target_len, teacher_forcing_prob=0.5):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.target_len = target_len
        self.teacher_forcing_prob = teacher_forcing_prob
        self.outputs = None

    def init_outputs(self, batch_size):
        device = next(self.parameters()).device
        # N, L (target), F
        self.outputs = torch.zeros(batch_size,
                              self.target_len,
                              self.encoder.n_features).to(device)

    def store_output(self, i, out):
        # Stores the output
        self.outputs[:, i:i+1, :] = out

    def forward(self, X, target_tensor=None):
        # X is batch of sentences -> N, F
        # splits the data in source and target sequences
        # the target seq will be empty in testing mode
        # N, L, F

        # Encoder expected N, F
        hidden_seq = self.encoder(X)
        # Output is N, F, hidden_dim
        self.decoder.init_hidden(hidden_seq)

        # The last input of the encoder is also
        # the first input of the decoder
        #dec_inputs = source_seq[:, -1:, :]
        batch_size = hidden_seq.size(0)

        dec_inputs = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_outputs = []
        # Generates as many outputs as the target length
        for i in range(self.target_len):
            # Output of decoder is N, 1, F
            decoder_output = self.decoder(dec_inputs)
            decoder_outputs.append(decoder_output)

            prob = self.teacher_forcing_prob

            # In evaluation/test the target sequence is
            # unknown, so we cannot use teacher forcing
            if not self.training:
                prob = 0

            if torch.rand(1) <= prob and target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                dec_inputs = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                dec_inputs = topi.squeeze(-1).detach()  # detach from history as input

            #_, topi = decoder_output.topk(1)
            #dec_inputs = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs

In [ ]:
full_seq = torch.full((2, 3), 1)
full_seq = full_seq.to(device)

In [ ]:
hidden_size = 128
batch_size = 32
encoder = EncoderSelfAttn(n_features=input_lang.n_words, d_model=hidden_size, n_heads=4, ff_units=4).to(device)
decoder = DecoderSelfAttn(n_features=output_lang.n_words, d_model=hidden_size, n_heads=4, ff_units=4).to(device)

In [ ]:
encdec = EncoderDecoderSelfAttention(encoder, decoder, target_len=10)
outputs = encdec(full_seq)
outputs

tensor([[[-8.1815, -7.6953, -7.7988,  ..., -7.8102, -8.5227, -8.0935],
         [-8.2636, -7.6932, -7.7879,  ..., -7.7972, -8.4904, -8.1159],
         [-8.2953, -7.7139, -7.8056,  ..., -7.9240, -8.4047, -8.1891],
         ...,
         [-8.2636, -7.6932, -7.7879,  ..., -7.7972, -8.4904, -8.1159],
         [-8.2953, -7.7139, -7.8056,  ..., -7.9240, -8.4047, -8.1891],
         [-8.2636, -7.6932, -7.7879,  ..., -7.7972, -8.4904, -8.1159]],

        [[-8.1815, -7.6953, -7.7988,  ..., -7.8102, -8.5227, -8.0935],
         [-8.2636, -7.6932, -7.7879,  ..., -7.7972, -8.4904, -8.1159],
         [-8.2953, -7.7139, -7.8056,  ..., -7.9240, -8.4047, -8.1891],
         ...,
         [-8.2636, -7.6932, -7.7879,  ..., -7.7972, -8.4904, -8.1159],
         [-8.2953, -7.7139, -7.8056,  ..., -7.9240, -8.4047, -8.1891],
         [-8.2636, -7.6932, -7.7879,  ..., -7.7972, -8.4904, -8.1159]]],
       grad_fn=<LogSoftmaxBackward0>)

# Transformer

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, max_len, d_model):
        super().__init__()
        self.d_model = d_model
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).float().unsqueeze(1)
        angular_speed = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * angular_speed) # even dimensions
        pe[:, 1::2] = torch.cos(position * angular_speed) # odd dimensions
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        # x is N, L, D
        # pe is 1, maxlen, D
        scaled_x = x * np.sqrt(self.d_model)
        encoded = scaled_x + self.pe[:, :x.size(1), :]
        return encoded # N, L, D

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, n_heads, d_model, dropout=0.1):
        super(MultiHeadedAttention, self).__init__()
        self.n_heads = n_heads
        self.d_model = d_model
        self.d_k = int(d_model / n_heads)
        self.linear_query = nn.Linear(d_model, d_model)
        self.linear_key = nn.Linear(d_model, d_model)
        self.linear_value = nn.Linear(d_model, d_model)
        self.linear_out = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(p=dropout)
        self.alphas = None

    def make_chunks(self, x):
        batch_size, seq_len = x.size(0), x.size(1)
        # N, L, D -> N, L, n_heads * d_k
        x = x.view(batch_size, seq_len, self.n_heads, self.d_k)
        # N, n_heads, L, d_k
        x = x.transpose(1, 2)
        return x

    def init_keys(self, key):
        # N, n_heads, L, d_k
        self.proj_key = self.make_chunks(self.linear_key(key))
        self.proj_value = self.make_chunks(self.linear_value(key))

    def score_function(self, query):
        # scaled dot product
        # N, n_heads, L, d_k x # N, n_heads, d_k, L -> N, n_heads, L, L
        proj_query = self.make_chunks(self.linear_query(query))
        dot_products = torch.matmul(proj_query,
                                    self.proj_key.transpose(-2, -1))
        scores =  dot_products / np.sqrt(self.d_k)
        return scores

    def attn(self, query, mask=None):
        # Query is batch-first: N, L, D
        # Score function will generate scores for each head
        scores = self.score_function(query) # N, n_heads, L, L
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        alphas = F.softmax(scores, dim=-1) # N, n_heads, L, L
        alphas = self.dropout(alphas)
        self.alphas = alphas.detach()

        # N, n_heads, L, L x N, n_heads, L, d_k -> N, n_heads, L, d_k
        context = torch.matmul(alphas, self.proj_value)
        return context

    def output_function(self, contexts):
        # N, L, D
        out = self.linear_out(contexts) # N, L, D
        return out

    def forward(self, query, mask=None):
        if mask is not None:
            # N, 1, L, L - every head uses the same mask
            mask = mask.unsqueeze(1)

        # N, n_heads, L, d_k
        context = self.attn(query, mask=mask)
        # N, L, n_heads, d_k
        context = context.transpose(1, 2).contiguous()
        # N, L, n_heads * d_k = N, L, d_model
        context = context.view(query.size(0), -1, self.d_model)
        # N, L, d_model
        out = self.output_function(context)
        return out

In [ ]:
watch1 = """
The Hatter was the first to break the silence. `What day of the month is it?' he said, turning to Alice:  he had taken his watch out of his pocket, and was looking at it uneasily, shaking it every now and then, and holding it to his ear.
"""

watch2 = """
Alice thought this a very curious thing, and she went nearer to watch them, and just as she came up to them she heard one of them say, `Look out now, Five!  Don't go splashing paint over me like that!
"""

sentences = [watch1, watch2]

## Preprocessing Embeddings

## Transformer Encoder

![](https://github.com/dvgodoy/PyTorchStepByStep/blob/master/images/enc_both.png?raw=1)

$$
\large
\begin{aligned}
&\text{outputs}_{\text{norm-last}}=&\text{norm}(\underbrace{\text{norm(inputs + att(inputs))}}_{\text{Output of SubLayer}_0} + \text{ffn}(\underbrace{\text{norm(inputs + att(inputs))}}_{\text{Output of SubLayer}_0}))
\\
\\
&\text{outputs}_{\text{norm-first}}=&\underbrace{\text{inputs + att(norm(inputs))}}_{\text{Output of SubLayer}_0}+\text{ffn(norm(}\underbrace{\text{inputs + att(norm(inputs))}}_{\text{Output of SubLayer}_0}))
\end{aligned}
$$

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, n_heads, d_model, ff_units, dropout=0.1):
        super().__init__()
        self.n_heads = n_heads
        self.d_model = d_model
        self.ff_units = ff_units
        self.self_attn_heads = MultiHeadedAttention(n_heads, d_model,
                                                    dropout=dropout)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, ff_units),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(ff_units, d_model),
        )

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.drop1 = nn.Dropout(dropout)
        self.drop2 = nn.Dropout(dropout)

    def forward(self, query, mask=None):
        # Sublayer #0
        # Norm
        norm_query = self.norm1(query)
        # Multi-headed Attention
        self.self_attn_heads.init_keys(norm_query)
        states = self.self_attn_heads(norm_query, mask) # N, L, D
        # Add
        att = query + self.drop1(states) # N, L, D

        # Sublayer #1
        # Norm
        norm_att = self.norm2(att) # N, L, D
        # Feed Forward
        out = self.ffn(norm_att) # N, L, D
        # Add
        out = att + self.drop2(out)
        return out # N, L, D

In [ ]:
class EncoderTransf(nn.Module):
    def __init__(self, encoder_layer, n_features, n_layers=1, max_len=100):
        super().__init__()
        self.d_model = encoder_layer.d_model
        self.n_features = n_features
        self.embd = nn.Embedding(self.n_features, self.d_model)
        self.pe = PositionalEncoding(max_len, self.d_model)
        self.norm = nn.LayerNorm(self.d_model)
        self.layers = nn.ModuleList([copy.deepcopy(encoder_layer)
                                     for _ in range(n_layers)])

    def forward(self, query, mask=None):
        # query is source sequence of shape N, L    i.e. N is the batch size, L is the sequence length
        X_embd = self.embd(query) # N, L -> N, L, D
        # Positional Encoding
        x = self.pe(X_embd) # N, L, D
        for layer in self.layers:
            x = layer(x, mask) # N, L, D
        # Norm
        return self.norm(x) # N, L, D

### Encoder transformer with preprocessed bert embeddings (Ignore for now)

In [ ]:
context_embeddings_tensor.size()

torch.Size([100, 768])

In [ ]:
# class MultiHeadedAttention(nn.Module):
#     def __init__(self, n_heads, d_ff, d_model, dropout=0.1):
#         super(MultiHeadedAttention, self).__init__()
#         self.n_heads = n_heads
#         self.d_model = d_model
#         self.d_ff = d_ff
#         self.d_k = int(d_model / n_heads)
#         self.linear_query = nn.Linear(d_model, d_model)
#         self.linear_key = nn.Linear(d_model, d_model)
#         self.linear_value = nn.Linear(d_model, d_model)
#         self.linear_out = nn.Linear(d_model, d_ff)
#         self.dropout = nn.Dropout(p=dropout)
#         self.alphas = None

#     def make_chunks(self, x):
#         batch_size, seq_len = x.size(0), x.size(1)
#         # N, L, D -> N, L, n_heads * d_k
#         x = x.view(batch_size, seq_len, self.n_heads, self.d_k)
#         # N, n_heads, L, d_k
#         x = x.transpose(1, 2)
#         return x

#     def init_keys(self, key):
#         # N, n_heads, L, d_k
#         self.proj_key = self.make_chunks(self.linear_key(key))
#         self.proj_value = self.make_chunks(self.linear_value(key))

#     def score_function(self, query):
#         # scaled dot product
#         # N, n_heads, L, d_k x # N, n_heads, d_k, L -> N, n_heads, L, L
#         proj_query = self.make_chunks(self.linear_query(query))
#         dot_products = torch.matmul(proj_query,
#                                     self.proj_key.transpose(-2, -1))
#         scores =  dot_products / np.sqrt(self.d_k)
#         return scores

#     def attn(self, query, mask=None):
#         # Query is batch-first: N, L, D
#         # Score function will generate scores for each head
#         scores = self.score_function(query) # N, n_heads, L, L
#         if mask is not None:
#             scores = scores.masked_fill(mask == 0, -1e9)
#         alphas = F.softmax(scores, dim=-1) # N, n_heads, L, L
#         alphas = self.dropout(alphas)
#         self.alphas = alphas.detach()

#         # N, n_heads, L, L x N, n_heads, L, d_k -> N, n_heads, L, d_k
#         context = torch.matmul(alphas, self.proj_value)
#         return context

#     def output_function(self, contexts):
#         # N, L, D
#         out = self.linear_out(contexts) # N, L, d_ff
#         return out

#     def forward(self, query, mask=None):
#         if mask is not None:
#             # N, 1, L, L - every head uses the same mask
#             mask = mask.unsqueeze(1)

#         # N, n_heads, L, d_k
#         context = self.attn(query, mask=mask)
#         # N, L, n_heads, d_k
#         context = context.transpose(1, 2).contiguous()
#         # N, L, n_heads * d_k = N, L, d_model
#         context = context.view(query.size(0), -1, self.d_model)
#         # N, L, d_ff
#         out = self.output_function(context)
#         return out # N, L, d_ff

In [ ]:
# class EncoderLayer(nn.Module):
#     def __init__(self, n_heads, d_ff, d_model, ff_units, dropout=0.1):
#         super().__init__()
#         self.n_heads = n_heads
#         self.d_ff = d_ff
#         self.d_model = d_model
#         self.ff_units = ff_units
#         self.self_attn_heads = MultiHeadedAttention(n_heads, d_model,
#                                                     dropout=dropout)
#         self.ffn = nn.Sequential(
#             nn.Linear(d_model, ff_units),
#             nn.ReLU(),
#             nn.Dropout(dropout),
#             nn.Linear(ff_units, d_ff),
#         )

#         self.norm1 = nn.LayerNorm(d_model)
#         self.norm2 = nn.LayerNorm(d_model)
#         self.drop1 = nn.Dropout(dropout)
#         self.drop2 = nn.Dropout(dropout)

#     def forward(self, query, mask=None):
#         # Sublayer #0
#         # Norm
#         norm_query = self.norm1(query)
#         # Multi-headed Attention
#         self.self_attn_heads.init_keys(norm_query)
#         states = self.self_attn_heads(norm_query, mask) # N, L, d_ff
#         # Add
#         att = query + self.drop1(states) # N, L, d_ff

#         # Sublayer #1
#         # Norm
#         norm_att = self.norm2(att) # N, L, D
#         # Feed Forward
#         out = self.ffn(norm_att) # N, L, d_ff
#         # Add
#         out = att + self.drop2(out)
#         return out # N, L, d_ff

In [ ]:
# class EncoderTransf(nn.Module):
#     def __init__(self, encoder_layer, pretrained_embeddings, n_features, n_layers=1, max_len=100):
#         super().__init__()
#         self.d_model = encoder_layer.d_model
#         self.embd_dim = pretrained_embeddings.size(-1)
#         self.n_features = n_features
#         #self.embd = nn.Embedding(self.n_features, self.d_model)
#         # Reshape the embeddings to (Vocabulary Size, Embedding Dimension)

#         # Create an Embedding layer and initialize it with the reshaped embeddings
#         self.embd = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=True)
#         self.pe = PositionalEncoding(max_len, self.d_model)
#         self.norm = nn.LayerNorm(self.d_model)
#         self.layers = nn.ModuleList([copy.deepcopy(encoder_layer)
#                                      for _ in range(n_layers)])

#     def forward(self, query, mask=None):
#         # query is source sequence of shape N, L    i.e. N is the batch size, L is the sequence length
#         X_embd = self.embd(query) # N, L -> N, L, D
#         # Positional Encoding
#         x = self.pe(X_embd) # N, L, D
#         for layer in self.layers:
#             x = layer(x, mask) # N, L, D
#         # Norm
#         return self.norm(x) # N, L, D

#### Testing Encoder with preprocessed embeddings

In [ ]:
# enclayer = EncoderLayer(d_model=context_embeddings_tensor.size(-1), n_heads=3, ff_units=20)
# enctransf = EncoderTransf(enclayer, pretrained_embeddings=context_embeddings_tensor, n_features=2, n_layers=1)

In [ ]:
# full_seq = torch.full((2, 2), 1)  # remove me
# outputs = enctransf(full_seq)
# outputs

tensor([[[ 0.7375, -1.1245, -0.1031,  1.8165, -0.6327, -0.6937],
         [ 0.8794, -1.3398, -0.2175,  1.6932, -0.5368, -0.4785]],

        [[ 0.5379, -1.1888, -0.1337,  1.9076, -0.6321, -0.4908],
         [ 0.9084, -1.2327, -0.1013,  1.6855, -0.5569, -0.7029]]],
       grad_fn=<NativeLayerNormBackward0>)

### Testing Transformer Encoder

In [ ]:
enclayer = EncoderLayer(d_model=6, n_heads=3, ff_units=20)
enctransf = EncoderTransf(enclayer, n_features=2, n_layers=1)

In [ ]:
#enclayer = nn.TransformerEncoderLayer(d_model=6, nhead=3, dim_feedforward=20)
#enctransf = nn.TransformerEncoder(enclayer, num_layers=1, norm=nn.LayerNorm)

## Transformer Decoder

![](https://github.com/dvgodoy/PyTorchStepByStep/blob/master/images/dec_both.png?raw=1)

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, n_heads, d_model, ff_units, dropout=0.1):
        super().__init__()
        self.n_heads = n_heads
        self.d_model = d_model
        self.ff_units = ff_units
        self.self_attn_heads = MultiHeadedAttention(n_heads, d_model,
                                                    dropout=dropout)
        self.cross_attn_heads = MultiHeadedAttention(n_heads, d_model,
                                                     dropout=dropout)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, ff_units),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(ff_units, d_model),
        )

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.drop1 = nn.Dropout(dropout)
        self.drop2 = nn.Dropout(dropout)
        self.drop3 = nn.Dropout(dropout)

    def init_keys(self, states):
        self.cross_attn_heads.init_keys(states)

    def forward(self, query, source_mask=None, target_mask=None):
        # Sublayer #0
        # Norm
        norm_query = self.norm1(query)
        # Masked Multi-head Attention
        self.self_attn_heads.init_keys(norm_query)
        states = self.self_attn_heads(norm_query, target_mask)   # N, 1, D
        # Add
        att1 = query + self.drop1(states)

        # Sublayer #1
        # Norm
        norm_att1 = self.norm2(att1)
        # Multi-head Attention
        encoder_states = self.cross_attn_heads(norm_att1, source_mask) # N, 1, D
        # Add
        att2 = att1 + self.drop2(encoder_states)

        # Sublayer #2
        # Norm
        norm_att2 = self.norm3(att2)
        # Feed Forward
        out = self.ffn(norm_att2) # N, 1, D
        # Add
        out = att2 + self.drop3(out)
        return out # N, 1, D

In [ ]:
class DecoderTransf(nn.Module):
    def __init__(self, decoder_layer, output_features, n_layers=1, max_len=100):
        super(DecoderTransf, self).__init__()
        self.d_model = decoder_layer.d_model
        self.embedding = nn.Embedding(output_features, self.d_model)
        self.pe = PositionalEncoding(max_len, self.d_model)
        self.norm = nn.LayerNorm(self.d_model)
        self.layers = nn.ModuleList([copy.deepcopy(decoder_layer)
                                     for _ in range(n_layers)])

    def init_keys(self, states):
        for layer in self.layers:
            layer.init_keys(states)

    def forward(self, query, source_mask=None, target_mask=None):
        # Embedding
        query = self.embedding(query) # N, 1 -> N, 1, D
        # Positional Encoding
        x = self.pe(query)
        for layer in self.layers:
            x = layer(x, source_mask, target_mask)
        # Norm
        return self.norm(x) # N, 1, D

## Transformer Encoder-Decoder

In [ ]:
class EncoderDecoderSelfAttentionTransformer(nn.Module):
    def __init__(self, encoder, decoder, output_features, target_len, teacher_forcing_prob=0.5):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.target_len = target_len
        self.teacher_forcing_prob = teacher_forcing_prob
        self.outputs = None
        self.linear = nn.Linear(encoder.d_model, output_features)

    def init_outputs(self, batch_size):
        device = next(self.parameters()).device
        # N, L (target), F
        self.outputs = torch.zeros(batch_size,
                              self.target_len,
                              self.encoder.n_features).to(device)

    def store_output(self, i, out):
        # Stores the output
        self.outputs[:, i:i+1, :] = out

    def encode(self, source_seq, source_mask=None):
        # Projection
        #source_proj = self.proj(source_seq)
        encoder_states = self.encoder(source_seq, source_mask)
        self.decoder.init_keys(encoder_states)
        return encoder_states

    def decode(self, shifted_target_seq, source_mask=None, target_mask=None):
        # Projection
        #target_proj = self.proj(shifted_target_seq)
        outputs = self.decoder(shifted_target_seq,
                               source_mask=source_mask,
                               target_mask=target_mask)
        # Linear
        outputs = self.linear(outputs)
        return outputs

    def forward(self, X, target_tensor=None):
        # X is batch of sentences -> N, L
        # splits the data in source and target sequences
        # the target seq will be empty in testing mode
        # N, L, D

        encoder_states = self.encode(X) # N, L, D
        # Encoder expected N, L
        #encoder_states = self.encoder(X)
        # Output is N, L, D
        #self.decoder.init_keys(encoder_states)

        # The last input of the encoder is also
        # the first input of the decoder
        #dec_inputs = source_seq[:, -1:, :]
        batch_size = encoder_states.size(0)

        dec_inputs = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_outputs = []
        # Generates as many outputs as the target length
        for i in range(self.target_len):
            # Output of decoder is N, 1, D
            decoder_output = self.decode(dec_inputs)
            #decoder_output = self.linear(decoder_output)
            decoder_outputs.append(decoder_output)

            prob = self.teacher_forcing_prob

            # In evaluation/test the target sequence is
            # unknown, so we cannot use teacher forcing
            if not self.training:
                prob = 0

            if torch.rand(1) <= prob and target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                dec_inputs = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                dec_inputs = topi.squeeze(-1).detach()  # detach from history as input

            #_, topi = decoder_output.topk(1)
            #dec_inputs = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs

### Testing Encoder-Decoder Transformer architecture

In [ ]:
full_seq = torch.full((2, 5), 1)

In [ ]:
enclayer = EncoderLayer(d_model=12, n_heads=3, ff_units=20)
enctransf = EncoderTransf(enclayer, n_features=input_lang.n_words, n_layers=1)

declayer = DecoderLayer(d_model=12, n_heads=3, ff_units=20)
dectransf = DecoderTransf(decoder_layer=declayer, output_features=output_lang.n_words, n_layers=1)

In [ ]:
enc_dec = EncoderDecoderSelfAttentionTransformer(enctransf, dectransf, output_features=output_lang.n_words, target_len=2)
enc_dec(full_seq).size()

torch.Size([2, 2, 2991])

In [ ]:
# Encoder-Decoder with Attention and RNN
#encoder = Encoder(input_lang.n_words, hidden_size).to(device)
#decoder = DecoderAttn(output_lang.n_words, hidden_size).to(device)
#model = EncoderDecoder(encoder, decoder, target_len=10)
# Transformer
enclayer = EncoderLayer(d_model=hidden_size, n_heads=3, ff_units=20)
enctransf = EncoderTransf(enclayer, n_features=input_lang.n_words, n_layers=1)
declayer = DecoderLayer(d_model=hidden_size, n_heads=3, ff_units=20)
dectransf = DecoderTransf(decoder_layer=declayer, output_features=output_lang.n_words, n_layers=1)
model = EncoderDecoderSelfAttentionTransformer(enctransf, dectransf, output_features=output_lang.n_words, target_len=10)

for data in train_dataloader:
    input_tensor, target_tensor = data
    decoder_outputs = model(input_tensor, target_tensor)
    print(f"target_tensor: {target_tensor.view(-1).size()}, decoder_outputs: {decoder_outputs.view(-1, decoder_outputs.size(-1)).size()}")
    break

IndexError: index out of range in self

# Train

In [ ]:
def train_epoch(dataloader, model, optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        optimizer.zero_grad()
        decoder_outputs = model(input_tensor, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        # Step 4 - Updates parameters using gradients and the learning rate
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def train(train_dataloader, model, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, model, optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
#@title Testing Encoder-Decoder with Attention and RNN on one batch of dataset
encoder = Encoder(input_lang.n_words, hidden_size).to(device)
decoder = DecoderAttn(output_lang.n_words, hidden_size).to(device)
model = EncoderDecoder(encoder, decoder, target_len=10)

for data in train_dataloader:
    input_tensor, target_tensor = data
    decoder_outputs = model(input_tensor, target_tensor)
    print(target_tensor.view(-1).size())
    print(decoder_outputs.size())
    print(f"target_tensor: {target_tensor.view(-1).size()}, decoder_outputs: {decoder_outputs.view(-1, decoder_outputs.size(-1)).size()}")
    break

In [ ]:
# Preprocessed embeddings dataloader
#input_lang, output_lang, train_dataloader, context_embeddings_tensor = get_dataloader(batch_size=32)

In [ ]:
# # Simple Encoder-Decoder with preprocessed embeddings

# encoder = Encoder(context_embeddings_tensor, hidden_dim=hidden_size).to(device)
# decoder = Decoder(pretrained_embeddings=context_embeddings_tensor, output_size=output_lang.n_words, hidden_dim=hidden_size).to(device)
# model = EncoderDecoder(encoder, decoder, target_len=10)

# for data in train_dataloader:
#     input_tensor, target_tensor = data
#     decoder_outputs = model(input_tensor, target_tensor)
#     print(target_tensor.view(-1).size())
#     print(decoder_outputs.size())
#     print(f"target_tensor: {target_tensor.view(-1).size()}, decoder_outputs: {decoder_outputs.view(-1, decoder_outputs.size(-1)).size()}")
#     break

In [ ]:
hidden_size = 126
batch_size = 32

#input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

# Dataloader for preprocessed embeddings
input_lang, output_lang, train_dataloader, context_embeddings_tensor = get_dataloader(batch_size=batch_size)
# Simple Encoder-Decoder with preprocessed embeddings
encoder = Encoder(context_embeddings_tensor, hidden_dim=hidden_size).to(device)
decoder = Decoder(pretrained_embeddings=context_embeddings_tensor, output_size=output_lang.n_words, hidden_dim=hidden_size).to(device)
model = EncoderDecoder(encoder, decoder, target_len=10)

# Encoder-Decoder with Attention and RNN (This is the best model so far)
#encoder = Encoder(input_lang.n_words, hidden_size).to(device)
#decoder = DecoderAttn(output_lang.n_words, hidden_size).to(device)
#model = EncoderDecoder(encoder, decoder, target_len=10)

# Encoder-Decoder Self-Attention
#encoder = EncoderSelfAttn(n_features=input_lang.n_words, d_model=hidden_size, n_heads=4, ff_units=4).to(device)
#decoder = DecoderSelfAttn(n_features=output_lang.n_words, d_model=hidden_size, n_heads=4, ff_units=4).to(device)
#model = EncoderDecoderSelfAttention(encoder, decoder, target_len=10)

# Transformer
# enclayer = EncoderLayer(d_model=hidden_size, n_heads=3, ff_units=20)
# enctransf = EncoderTransf(enclayer, n_features=input_lang.n_words, n_layers=1)
# declayer = DecoderLayer(d_model=hidden_size, n_heads=3, ff_units=20)
# dectransf = DecoderTransf(decoder_layer=declayer, output_features=output_lang.n_words, n_layers=1)
# model = EncoderDecoderSelfAttentionTransformer(enctransf, dectransf, output_features=output_lang.n_words, target_len=10)

train(train_dataloader, model, 80, print_every=5, plot_every=5)

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
worked so far


RuntimeError: Expected hidden size (1, 32, 126), got [1, 2, 128]

In [ ]:
def evaluate(model, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        decoder_outputs = model(input_tensor)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words

In [ ]:
model.eval() # Set to evaluation mode

In [ ]:
n = 10
print("French to English:\n")
for i in range(n):
    pair = random.choice(pairs)
    print(f"{pair[0]} -> {' '.join(evaluate(model, pair[0], input_lang, output_lang))}, CORRECT TRANSLATION: {pair[1]}")

In [ ]:
' '.join(evaluate(model, 'je vais bien', input_lang, output_lang))

### _Note: I have not included < unk >. If word does not exist already in the embedding it will throw an error._